# Use Case 1: Defibrillator data from out-of-hospital cardiac arrest case

<span style = "font-size:20px">This notebook illustrates the usage of the `vitabel` class to visualize, annotate and process time-series data from the medical field. In this case we analyze data recorded by a defibrillator during a real-world out-of-hospital cardiac arrest case.<br>
This notebook in particular demonstrates the capabilities of vitabel to **display** and **annote** data.
</span>


In [ ]:
from vitabel import Vitals, Label

import bz2
from pathlib import Path
import tempfile
import shutil
from datetime import datetime
import pandas as pd

## 1) Load Data

<span style = "font-size:18px">An empty vitabel object called `case` is initiated and all channels from a defibrillator recording (in this case a ZOLL X-Series case, stored in a compressed file) are loaded and added to the case.</span>

In [ ]:
case = Vitals()

compressed_defi_file = Path("data/ZOLL_test_case.json.bz2")
with tempfile.TemporaryDirectory() as tmpdir:
    defi_file = Path(tmpdir) / compressed_defi_file.stem
    with bz2.open(compressed_defi_file, "rb") as source:
        with open(defi_file, "wb") as dest:
            shutil.copyfileobj(source, dest)

    case.add_defibrillator_recording(defi_file)

<span style = "font-size:18px">Every vitable object as well as every channel and label holds a dictionary as `metadata`. In the following we add the usage of the case in this notebook to the metadata. </span>

In [ ]:
usage_metadata = {
    "vitable publication" : {"user" : "ENTER YOUR NAME HERE",
                             "purpose" : "higlighting capabilities to display and annotate defibrillator data",
                             "remarks" : ["a usefull tool for researchers", "handling data of devices by other manufacturers should be integrated better in the future"]
                            }
}
case.metadata.update({"usage":usage_metadata})

print("\nThis are the current metadata stored:\n")
case.metadata

<span style = "font-size:18px">We get an overview over all loaded channels by calling `case.info()`</span>

In [ ]:
case.info()

## 2) Process Data

<span style = "font-size:18px">We use integrated algorithms to extract **etCO₂** values and detect **ventilations** from the capnography signal.\
Additionally, we analyze the accelerometer data from the CPR feedback device and the ECG signal from the defibrillation pads to estimate the **probability of spontaneous circulation**.</span>

In [ ]:
case.compute_etco2_and_ventilations()
case.predict_circulation()

## 3) Define new Label

<span style = "font-size:18px">We aim to manually **annotate** occurrences of **Return of Spontaneous Circulation (ROSC)**.\
Therefore we create an empty label and add it to the case. With `metadata` we can store miscellaneous data in the label inside a dictionary (`{}`).\
Further we can define the `plotstyle` by giving parameters in another dictionatry.\
<br>
`plot_type` defines the way data of the label are depicted in the plot. As the type is also dynamically adapted to the availabel data, we can explore the different types by adding labels with different data in the later generated plot.\
This can be done by activating and deactivating the checkboxes in various combinations.\
<br>
More details on Labels can be found [here](https://vitabel.readthedocs.io/en/latest/autoapi/vitabel/timeseries/index.html#vitabel.timeseries.Label).
</span>

In [ ]:
ROSC_label = Label(
    name = "ROSC",
    time_index = [],
    data = [], 
    text_data = [], 
    metadata = {
        "Source": "manual annotation",
        "time_of_generation" : datetime.now()
    },
    plot_type = "combined",
    plotstyle = {"marker": "$\u2665$", "color": "red", "ms": 10, "linestyle": ""},

)

case.add_global_label(ROSC_label)

<span style = "font-size:18px">We see the newly added label (`ROSC`) by checking out all present labels in the case.</span>

In [ ]:
case.get_label_names()

## 4) Interactively Plot Data

<span style = "font-size:18px">We provide a setup for the interactive plot.\
The `channels` to be plottted are defined as lists in a list (`[[],[]]`). Each of the inner lists define a new subplot.\
The `labels` to be plotted on top are defined by another lists in a list.\
Subplots below the already defined ones are defined by `channel_overviews`. These subplots serve a special purpose by displaying the entire recording interval and highlighting the depicted subsegement in the upper subplots.\
<br>
Above the plot a menu is given to `Annotate`, `Align Timelines`, and deffines `Settings` of the plot.

</span>

In [ ]:
plot = case.plot_interactive(
    channels = [
        ["cpr_acceleration"], 
        ["capnography"], 
        ["ecg_pads"], 
        []
    ],
    labels = [
        ["ROSC"],
        ["etco2_from_capnography", "ROSC"],
        ["ROSC"],
        ["ROSC", "rosc_probability"],
    ],
    channel_overviews = [["cpr_acceleration"]],
    time_unit = "s",
    subplots_kwargs = {"figsize": (16.5, 8)},
)
plot

<span style = "font-size:18px">Show the added `data` to the ROSC label</span>

In [ ]:
ROSC_label.get_data()

## 5) Store Data
<span style = "font-size:18px">The entire case can be sotred as a json file.</span>


In [ ]:
case.save_data("case_1.json")

<span style = "font-size:18px">Data of the added label can also be stored as a csv spreadsheet.</span>

In [ ]:
case.get_label("ROSC").to_csv()
pd.read_csv("ROSC.csv", index_col = 0)

<span style = "font-size:18px">Alternatively the added label can be returned as dictionary with all its features.</span>


In [ ]:
case.get_label("ROSC").to_dict()